In [2]:
from operator import itemgetter
from collections import Counter, namedtuple
from itertools import chain
import re
import pdb
import pickle
import csv
from resources import sdf  # Dictionary of PGC ranges
import codecs
import datetime

TID = 'BHLHPABEK31'
# Load up the TIRKS data and create namedtuples...
with open('dcs.pickle', 'rb') as lf:
    lo_dcs = pickle.load(lf)
with open('dcs_names.pickle', 'rb') as lf:
    dcs_names = pickle.load(lf)
with open('st1.pickle', 'rb') as lf:
    lo_st1 = pickle.load(lf)
with open('st1_names.pickle', 'rb') as lf:
    st1_names = pickle.load(lf)
with open('t3z.pickle', 'rb') as lf:
    lo_t3z = pickle.load(lf)
with open('t3z_names.pickle', 'rb') as lf:
    t3z_names = pickle.load(lf)
DCS = namedtuple('DCS', dcs_names)
ST1 = namedtuple('ST1', st1_names)
T3Z = namedtuple('T3Z', t3z_names)
# populate the dictionaries of namedtuples
dcs = {}
for d in lo_dcs:
    dcs[d[3] + d[4]] = DCS(*d)
st1 = {}
for s in lo_st1:
    st1[s[1]] = ST1(*s)
t3z = {}
for t in lo_t3z:
    t3z[t[1]] = T3Z(*t)

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
def dcs_circuit(slot):
    """
    Return either the child_circuit or assignment circuit number along with which is being used (C or A)
    :slot: DCS slot index
    :return: Child_circuit or Assignment (C or A) and circuit name
    """
    try:
        assert dcs[slot].child_circuit, f'No DCS child_circuit for DCS: {slot}'
        return 'C', dcs[slot].child_circuit
    except AssertionError as error:
        pass
    try:
        assert dcs[slot].assignment, f'No DCS assignment for DCS: {slot}'
        return 'A', dcs[slot].assignment
    except AssertionError as error:
        pass
    return ''

In [4]:
def st1_match(ckt):
    return [st1[x] for x in st1 if st1[x].ckt_id == ckt]

def t3z_match(ckt):
    return [t3z[x] for x in t3z if t3z[x].ckt_id == ckt]
    
def dcs_match(slot, asgn, ckt):
    return [dcs[x] for x in dcs if (dcs[x].child_circuit == ckt or dcs[x].assignment == ckt) and dcs[x].slot != dcs[slot].slot]

In [5]:
def level_up(tab, matched):
    """
    Return the level 0 ckt_id for a matched child circuit
    :tab: the child_circuit tab
    :matched: the matching child_cicuirt namedtuple
    :return: ckt of  the level 0 circuit ex. child 22.10 (lvl=1) would return 2. (lvl=0)
    """    
    if tab == 'ST01':
        level = matched[0].item.split('.')
        return st1[level[0] + '.'].ckt_id, str(int(matched[0].chanwkg))
    if tab == 'T3Z':
        level = matched[0].item.split('.')
        return t3z[level[0] + '.'].ckt_id, str(int(matched[0].chanwkg))

In [6]:
def look_back(asgn, ckt):
    return  [dcs[x] for x in dcs if dcs[x].assignment == ckt]

In [7]:
subdiv_ex = []
def reformat_slot(slot, subdiv):
    if len(subdiv) == 3:
        return f'{slot[1:]}-{subdiv[0]}T-{subdiv[1]}-{subdiv[2]}'
    elif len(subdiv) == 2:
        return f'{slot}-{subdiv[0]}T-{subdiv[1]}'
    else:
        subdiv_ex.append([slot,subdiv])

In [8]:
def parse_xcon(xcon):
    if xcon.subdiv:
        return [repr(xcon.__class__)[-5:-2], reformat_slot(xcon.slot, xcon.subdiv), xcon.status, xcon.a_clli, xcon.z_clli, xcon.assignment, xcon.child_circuit]
    else:
        return [repr(xcon.__class__)[-5:-2], xcon.slot, xcon.status, xcon.a_clli, xcon.z_clli, xcon.assignment, xcon.child_circuit]

In [10]:
xcon = []
for slot in dcs:
    matched_st1 = []
    matched_t3z = []
    matched_dcs = []
    thing_1 = []
    
    asgn, ckt = dcs_circuit(slot)  # Is the dcs circuit an assignment or a child
    matched_st1 = st1_match(ckt)
    matched_t3z = t3z_match(ckt)

    if matched_st1:
        lvl0_ckt, chanwkg = level_up('ST01', matched_st1)
        thing_1 = look_back(asgn, lvl0_ckt)
        try:
            temp = dict(thing_1[0]._asdict())
            temp['slot'] += f'-{chanwkg}'
            thing_1 = [DCS(*temp.values())]
        except:
            pass

    if matched_t3z:
        lvl0_ckt, chanwkg = level_up('T3Z', matched_t3z)
        thing_1 = look_back(asgn, lvl0_ckt)
        try:
            temp = dict(thing_1[0]._asdict())
            temp['slot'] += f'-{chanwkg}'
            thing_1 = [DCS(*temp.values())]
        except:
            pass

    if not matched_st1 and not matched_t3z:
        matched_dcs = dcs_match(slot, asgn, ckt)
        
    xcon.append([dcs[slot], matched_st1, matched_t3z, thing_1, matched_dcs])

In [14]:
header = ['Type', 'Slot', 'Status', 'A CLLI', 'Z CLLI', 'Assignment', 'Child Circuit',
          'Child Type', 'Matched Circuit', 'Child A CLLI', 'Child Z CLLI',
          'Type', 'Slot', 'Status', 'A CLLI', 'Z CLLI', 'Assignment', 'Child Circuit']

filename = f'{TID}_TIRKS_Circuit_Mapping_{datetime.datetime.now().strftime("%y%m%d_%H%M")}.csv'
with open(filename, 'w') as tf:
    writer = csv.writer(tf)
    writer.writerow(header)
    for x in xcon:
        row = []
        if x[1]:
            for s in x[1]:
                row.extend(parse_xcon(x[0]))
                row.extend([repr(s.__class__)[-5:-2], s.ckt_id, s.aclli, s.zclli])
                if x[3]:
                    row.extend(parse_xcon(x[3][0]))
                if x[4]:
                    row.extend(parse_xcon(x[4][0]))
                writer.writerow(row)
                row = []

        if x[2]:
            for t in x[2]:
                row.extend(parse_xcon(x[0]))
                row.extend([repr(t.__class__)[-5:-2], t.ckt_id, t.aclli, t.zclli])
                if x[3]:
                    row.extend(parse_xcon(x[3][0]))
                if x[4]:
                    row.extend(parse_xcon(x[4][0]))
                writer.writerow(row)
                row = []
        
        if not x[1] and not x[2]:
            row.extend(parse_xcon(x[0]))
            if x[4]:
                if x[0].child_circuit:  # was always using assignment - fixed 09/29/18 18:10 
                    row.extend(['', x[0].child_circuit, '', ''])
                    row.extend(parse_xcon(x[4][0]))
                else:
                    row.extend(['', x[0].assignment, '', ''])
                    row.extend(parse_xcon(x[4][0]))
            writer.writerow(row)
            row = []

In [ ]:
        if not x[1] and not x[2]:
            row.extend(parse_xcon(x[0]))
            if x[4]:
                row.extend(['', x[0].assignment, '', ''])
                row.extend(parse_xcon(x[4][0]))
            writer.writerow(row)
            row = []